# Design Document for P0P

Partner A: Duncan Huntsinger

Partner B: Daimu Iwata

## 1. Thread-based Server Design
Implemented by: Duncan Huntsinger

#### Overview
In the cell below, describe the overall server design. You must include the number of threads you use including the main thread, what each thread is for and doing, any loops you have for each thread.

For the threaded server, I designed it so that threads could perform tasks as independently as possible so as to avoid loss of efficiency over synchronization. Each thread performed a small but monolothic task. One thread (the main thread) processed user input. Another performed all of the socket I/O and pushed raw bytes into a thread-safe queue as a nonblocking operation. Another thread processed the bytes from the queue into packets, performing correctness checks and taking the packets to the correct session for processing and handling responses plus errors. Timer threads would be scheduled once per each session every time a message was received. Lastly, there was a thread that popped logs (which were just tuples that would be splatted into print) pushed in by other threads and printed them to stdout. All of these functions except for timers operated within while loops, which checked a shared boolean variable that indicated whether or not the server was in a closing state. For all threads operating on queues, a sentinel value of `None` was pushed into their respective queues, with an if statement to check if such a value was emplaced into a queue. In total, this yield 4+ threads, though usually it's just 4 executing at the same time.

#### Justification 
In the cell below, justify why your design is efficent and show evidence it can handles load efficiently. Specify the number of clients, the file size for each client, and the response rate/delay each client.

My design is efficient as it does a fairly good job of splitting different categories of tasks (different I/O endpoints, data locations) into largely single-threaded control flows. I've included a script called `test.sh`. If you run the command `./server 7877` in one terminal, and `time bash test.sh` in another, you can get the approximate time it takes for the server to fully process 5 Dostoyevsky clients. These are the following times I got running on localhost five times:

1. 34.850s
2. 35.473s
3. 33.721s
4. 32.996s
5. 33.683s

As can be seen here, these times are barely above the 30s timeout for server.exp, and far below the 60s timeout mentioned on Piazza.


#### Comparing with Your Partner's Event-loop-based Server
Compared to your partner's Event-loop-based server, does your server approach use fewer/more/the same number of threads? Explain and justify.

My server uses more threads, since due to nonblocking I/O, my partner's writes didn't require much work on the end of individual threads, while mine involved synchronous system calls. Therefore to keep my server's performance roughly even with his, I needed to create kernel threads for all I/O tasks, not just reading.

#### Data structures
In the cell below, list any notable data structures you used and justify the use of the data structure. Also specify how you handle synchronization for the data structure if there were any need for synchronization.

These were the following data structures that I used:

- **queue.Queue**
    This was the bread and butter of thread communication for my server, working much like an assembly line for implementing P0P. A thread would take data, transform      it, then supply it to the next thread and immediately processing the next available piece of data. The nonblocking put and blocking get methods were perfect for       implementing this multithreading scheme.
- **threading.Lock**
    This was important for synchronizing the sessions hash table, as both the packet processing thread and the timer thread could both multiplex this field. 
- **dict**
    This was used for mapping session IDs (ints) to session metadata objects.

#### How timeouts are handled
In the cell below, decribe how the timeouts are handled.

Unlike the timer thread that Dr. Han had suggested, I decided to go with the `threading.Timer` class and using instances of that which would be scheduled after a time period. My reasoning for this is that while this would take up more memory, it would provide much higher throughput for high activity where these threads would be very unlikely to be scheduled, as opposed to a timer thread constantly looping over an array of session objects.

#### How shutdown is handled
In the cell below, describe how you handled the shutdown gracefully. (That is when you hit 'q' or ctrl+c in the command prompt.)

For handling the shutdown, I used a shared boolean for indicating whether or not the server was in the 'closing' state. Since booleans are builtin types, and evaluating booleans is atomic in Python due to the global interpreter lock, checking if the close state boolean is set is thread safe. For threads that used queues, I inserted `None`, which they knew to interpret as an end of life value to receive from a queue. I then iterated through the sessions hash map and called the close method, which would clean up the timer objects. I then closed the socket using the shutdown method. Although UDP sockets are connectionless and can't actually 'shut down', for some reason the idiomatic close method doesn't wake up sleeping threads, so I have to surround the shutdown call with a try-except block. I lastly make sure to join all of the non-timer threads.

#### Any libraries used
In the cell below, list any libraries that you used for the implementation and justify the usage.

- `sys` Used for accessing command line arguments.
- `threading` Used for concurrency.
- `socket` Used for network I/O.
- `queue` Used for thread-safe queue.
- `typing` Used for type annotations and readability.
- `lib` This was a library module I wrote since there was some common functionality shared between the server and the client, namely the packet format.

## 2. Event-loop-based Client Design
Implemented by: Duncan Huntsinger

#### Overview
In the cell below, describe the overall server design. You must include the number of threads you use including the main thread, what each thread is for and doing, any loops you have for each thread.

My event-loop-based client uses an object of type `P0PSession`, which inherits from the type `asyncio.DatagramProtocol`. This superclass provides a useful callback in the form of `datagram_received`, which handles the dirty work for receiving packets from a nonblocking socket (called a Transport in asyncio). Tasks are created for reading from standard input and timeouts and waited upon in the input loop. A task is also created for sending data, as while writes are nonblocking and don't have any result to yield for, as a client we are obligated to wait for a response from the server or time out if the server doesn't respond within ten seconds. We can spend this time processing subsequent user input, and if it turns out we've timed out by the time we've received user input, we can simply discard it. In a sense, it works similarly to branch prediction. In total, this yields three tasks/threads.

#### Events
In the cell below, list all events the thread that handles the event-loop are listening too. 

- Receive packet event
- Condition variable notify event
- Semaphore up event
- User input event

#### Justification 
In the cell below, justify why your design above is efficent. Also show evidence how your client can handle sending packets from a large file (each line is close to UDP packet max and also contains many lines) and receiving packets from the server at the same time. Note you do not want to cause the false TIMEOUT from server because you are too busy just sending out the packets to the server when the server actually has sent you a packet before the TIMEOUT.

Coroutines and nonblocking I/O are inherently more efficient since they don't use blocking/synchronous system calls, and as such don't rely on kernel threads, which greatly reduces their overhead. Additionally, the concept of coroutines is conducive to well structured, single threaded control flows. With this in mind, an asynchronous client using the same design philosophy I described when writing about how I designed my server to run efficiently would intrinsically be faster. I wasn't able to test messages close to the UDP max since readline() only handles up to 4096 bytes of input, but since the receive callback and sendto() operate in different tasks, they absolutely can handle writing and receiving packets to and from the server at the same time.

#### Comparing with Your Partner's Thread-based Client
Compared to your partner's Thread-based client, does your client use fewer/more/the same number of threads? Explain and justify.

My partner also uses three threads, since the specification of P0P requires that the client behave as if it were blocking (waiting for server response).

#### Data structures
In the cell below, list any notable data structures you used and justify the use of the data structure. Also specify how you handle synchronization for the data structure if there were any need for synchronization.

- **asyncio.DatagramTransport** Nonblocking UDP socket
- **asyncio.StreamReader, asyncio.StreamWriter** Nonblocking stdin and stdout streams, respectively
- **asyncio.Lock** Used for ensuring the atomicity of incrementing the session number
- **asyncio.Semaphore** Used to implement a rendezvous pattern in sendto() for commands that needed a response (HELLO, DATA)
- **asyncio.Condition** Used to interrupt input processing in the main task

#### How timeouts are handled
In the cell below, decribe how the timeouts are handled.

Timeouts are handled using the `asyncio.wait_for()` function. This function takes a future to wait on and a duration to wait before a timeout exception is raised. It allows for timing out to be implemented fairly easily.

#### How shutdown is handled
In the cell below, describe how you handled the shutdown gracefully. (That is when you hit 'q' or ctrl+c in the command prompt.)

I would loop through all of my tasks and cancel them, then await them to receive their cancellation error. I would also release the semaphores and close the transport object. Ctrl+C was handled using asyncio's signal handler framework, while q/EOF were handled as normal break conditions.

#### Any libraries used
In the cell below, list any libraries that you used and justify the usage.

- **sys** Getting command line arguments
- **socket** Function for resolving hostname
- **signal** Signal macro definitions
- **random** Generating session ID
- **asyncio** Concurrency and nonblocking I/O
- **lib** Same as in the server
- **typing** Same as in the server

## 3. Event-loop-based Server Design
Implemented by: Daimu Iwata

#### Overview
In the cell below, describe the overall server design. You must include the number of threads you use including the main thread, what each thread is for and doing, any loops you have for each thread.

In my Event-loop-based Server, I use one main thread that includes 2 event loops. One loops and waits until receiving a packet from a client. The other one loops and waits for user (of server) inputs. The code creates an object of type `P0PServer` which inherits `asyncio.DatagramProtocol` and I implemented `connection_made()` and `datagram_received()` functions as the protocol requires. Once a connection is made, `connection_made()` is called exanctly once and after that, every time my server receives a message, `datagram_received` is called which creates a new task to process the packet. If the received message is from a new client, my server creates a new session `P0PSession` to store the information of the client. When my server sends back to the client after receiving a message, it sets a timer (creating timer object of type `Timer`) with 10 seconds with the callback set as `_goodbye()` which sends GOODBYE message to the client and closes the session. And for the user input event loop, when the user hits q, it moves to the closing state and creates a new task `exit()` so that it avoids immediately closing the loop right after cancelling all the other tasks of the loop.

#### Events
In the cell below, list all events the thread that handles the event-loop are listening too. 

- timeout event
- user-input event
- packet receiving event
- exit (server closing) event

#### Justification 
In the cell below, justify why your design is efficent and show evidence it can handles load efficiently. Specify the number of clients, the file size for each client, and the response rate/delay each client.

The design is effeicient as it does not block the system. When the server is waiting for the user input or receiving a packet from a client, it yields to the other remaining tasks without blocking. When I tested my server with 5 of my partner's event-loop client with Dostoyevsky.txt for each, either of my server or each client did not timeout and finished sending the texts almost at the same time for each client.

#### Comparing with Your Partner's Thread-based Server
Compared to your partner's Thread-based server, does your server use fewer/more/the same number of threads? Explain and justify.

Compared to my partner's thread-based server, my server uses fewer number of threads as mine has only one main thread with some event loops. Since thread-based server creates some more threads to handle the I/O's, it slows down a little as it calls the system calls and it blocks the thread when it waits for user inputs or receiving a packet.

#### Data structures
In the cell below, list any notable data structures you used and justify the use of the data structure. Also specify how you handle synchronization for the data structure if there were any need for synchronization.

- **asyncio.DatagramTransport** nonblocking datagram transport
- **asyncio.streamReader** nonblocking stdin stream
- **asyncio.streamWriter** nonblocking stdout stream
- **dict** dictionary to map from the session ID to the object of P0PSession to check if the session has already created or not.

#### How timeouts are handled
In the cell below, decribe how the timeouts are handled.

Timeouts are handled by `Timer` class object, which includes a coroutine function `_timeout_start()` and `_cancel()`. Each session has only one timer, and when the server sends a message to the cient, it creates a Timer object and creates a task (coroutine) `_timeout_start()` setting 10 seconds as TIMEOUT. The implementation of `_timeout_start()` is that it sleeps for a certain timeout seconds (10 seconds) and calls a callback (goodbye) after waking up. When the server receives a message within 10 seconds, it cancels the timer by cancelling the task. 

#### How shutdown is handled
In the cell below, describe how you handled the shutdown gracefully. (That is when you hit 'q' or ctrl+c in the command prompt.)

When the user hits 'q' or ctrl+c, the stdin event loop calls `_close()` function which sends goodbye message to all the clients, and calls the global function `ask_exit()`. Since it causes a problem when I close the loop immediately right after cancelling the remaining tasks, it creates a new closing task after cancelling the remaining tasks. In the `exit()` coroutine, it just stops the loop and shutdowns gracefully.

#### Any libraries used
In the cell below, list any libraries that you used for the implementation and justify the usage.

- **sys** To get arguments from command prompt
- **asyncio** To implement nonblocking I/O UDP server and for timer sleep 
- **lib** a library module that my partner wrote. It handles the P0P packet and checks if it is valid or not
- **typing** To use some datastructures: Dict, Tuple, Union

## 4. Thread-based Client Design
Implemented by: Daimu Iwata

#### Overview
In the cell below, describe the overall server design. You must include the number of threads you use including the main thread, what each thread is for and doing, any loops you have for each thread.

My thread-based client includes 3 threads including the main thread. The main thread handles receiving messages from the connected server. Another thread handles the user input and sends the input message to the server. The last one handles timer for timeout event. My client first creates an object of type `P0PSession` and it loops to get the user input to send to the server until the user hits 'q' or ctrl+c, or the server terminates itself. When the server replies back, the receiving thread handles the message and check its validity. 

#### Justification 
In the cell below, justify why your design is efficent and show evidence how your client can handle sending packets from a large file (each line is close to UDP packet max and also contains many lines) and receiving packets from the server at the same time. (You do not want to cause the false TIMEOUT from server because you are too busy just sending out the packets to the server when the server actually has sent you a packet before the TIMEOUT.)

My thread-based client design is efficient as it could send the 10,000 lines of 65,000 characters without TIMEOUT event occurs. The actual time it took for the client to finish sending those to my partner's thread-based server was 89.8097... seconds. So that means my client was not too busy to send the characters as it could receive replies from the server and cancel the timers.

#### Comparing with Your Partner's Event-loop-based Client
Compared to your partner's Event-loop-based client, does your client use fewer/more/the same number of threads? Explain and justify.

Compared to my partner's event-loop client, mine uses more number of threads as his is non-blocking I/O which only uses one main thread to deal with the I/O tasks.  

#### Data structures
In the cell below, list any notable data structures you used and justify the use of the data structure. Also specify how you handle synchronization for the data structure if there were any need for synchronization.

- **threading.Timer** To set a timer for the timeout event

#### How timeouts are handled
In the cell below, decribe how the timeouts are handled.

I use `threading.Timer` class to handls the timeout events. My client holds only one timer object and when it sends an inputted data to the server from the "Ready" state, it creates a `threading.Timer` object with the TIMEOUT (10) seconds passed as an argument and the callback function `_close()` passed. When my client receives a reply ALIVE message or HELLO from the server, it cancels the timer.

#### How shutdown is handled
In the cell below, describe how you handled the shutdown gracefully. (That is when you hit 'q' or ctrl+c in the command prompt.)

When the user hits 'q' or ctrl+c, it moves to the closing state. It first sends the goodbye message to the server and set the timer. If the server replies with GOODBYE message, it'll cancel the timer and close the socket getting out of the while loop to shutdown gracefully. If the server does not reply, it'll resend the GOODBYE message and repeats the step until the server replies with GOODBYE. Since the main thread handles the packet receiving which catches exceptions (ctrl+c and timeout error) and the other threads are set daemon as True, When the main thread finishes, the other threads are also closed. 

#### Any libraries used
In the cell below, list any libraries that you used and justify the usage.

- **sys** To get arguments from command prompt
- **threading** To implement multi-threaded blocking I/O and timer thread.
- **random** To get a random session ID
- **socket** To use network I/O
- **lib** a library module that my partner wrote. It handles the P0P packet and checks if it is valid or not
- **typing** To use tuple and union

## Submission 

**Remember to "Save the Notebook" (from the "File" menu above). You can "Shutdown" to close Jupyter-lab **

Push the updated notebook to your github team repo under the project root (the same level as README.txt)